<a href="https://www.kaggle.com/code/scr0ll0/modeling-pii-data-detection?scriptVersionId=160176725" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

We will be using Presidio: a Microsoft framework for PII detection. Credit to pjmathematician for both the idea and the baseline code: https://www.kaggle.com/code/pjmathematician/pii-eda-presidio-baseline

# Downloading and Loading Libraries

In [1]:
%%capture
!pip install presidio_analyzer --no-index --find-links=file:///kaggle/input/presidio-wheels/presidio

In [2]:
import json
import pandas as pd
from tqdm.auto import tqdm
from presidio_analyzer import AnalyzerEngine, EntityRecognizer, PatternRecognizer, Pattern, RecognizerResult
from presidio_analyzer.context_aware_enhancers import LemmaContextAwareEnhancer
from presidio_analyzer.nlp_engine import NlpArtifacts, NlpEngineProvider
from presidio_analyzer.predefined_recognizers import PhoneRecognizer
from presidio_analyzer.recognizer_registry import RecognizerRegistry
import spacy
from dateutil import parser

# Loading Data

In [3]:
train = json.load(open('/kaggle/input/pii-detection-removal-from-educational-data/train.json'))
test = json.load(open('/kaggle/input/pii-detection-removal-from-educational-data/test.json'))

# Functions

In [4]:
#Functions sourced from pjmathematician and leonshangguan:
#https://www.kaggle.com/code/pjmathematician/pii-eda-presidio-baseline
#https://www.kaggle.com/code/leonshangguan/modify-of-pii-detect-study

def tokens2index(row):
    tokens  = row['tokens']
    start_ind = []
    end_ind = []
    prev_ind = 0
    for tok in tokens:
        start = prev_ind + row['full_text'][prev_ind:].index(tok)
        end = start+len(tok)
        start_ind.append(start)
        end_ind.append(end)
        prev_ind = end
    return start_ind, end_ind

In [5]:
def find_or_next_larger(arr, target):
    left, right = 0, len(arr) - 1

    while left <= right:
        mid = (left + right) // 2

        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            left = mid + 1
        else:
            right = mid - 1
    return left

In [6]:
def count_trailing_whitespaces(word):
    return len(word) - len(word.rstrip())

In [7]:
def is_valid_date(text):
    try:
        parsed_date = parser.parse(text)
        return True
    except:
        return False

# Modeling

In [8]:
ALLOW_LIST = ['phone', 'number', 'telephone', 'cell', 'cellphone',
              'mobile', 'call', 'ph', 'tel', 'mobile', 'Email']

DENY_LIST_LEON = ["wikipedia", "coursera", ".pdf", ".PDF", "article",
             ".png",".gov", ".work", ".ai", ".firm", ".arts",
             ".store", ".rec", ".biz", ".travel"]

DENY_LIST_EMAIL = []
DENY_LIST_ADDRESS = []
DENY_LIST_URL = []

In [9]:
#"Training"
#Iterate through all of the data, identify tokens that are labels, add to deny list

tokens = [j for i in train for j in i['tokens']]
labels = [j for i in train for j in i['labels']]

for i in set(labels):
    if i == 'O':
        continue
    elif i in ['B-EMAIL', 'B-STREET_ADDRESS', 'I-STREET_ADDRESS', 'B-URL_PERSONAL', 'I-URL_PERSONAL']:
        indices = [j for j in range(len(labels)) if labels[j] == i]
        if i == 'B-EMAIL':
            DENY_LIST_EMAIL.extend([tokens[i] for i in indices])
        elif i in ['B-STREET_ADDRESS', 'I-STREET_ADDRESS']:
            DENY_LIST_ADDRESS.extend([tokens[i] for i in indices])
        else:
            DENY_LIST_URL.extend([tokens[i] for i in indices])
    else:
        continue

In [10]:
#Credit to leonshangguan for the Pattern Recognizers: https://www.kaggle.com/code/leonshangguan/modify-of-pii-detect-study/notebook?scriptVersionId=159973328 

address_regex = r'\b\d+\s+\w+(\s+\w+)*\s+((st(\.)?)|(ave(\.)?)|(rd(\.)?)|(blvd(\.)?)|(ln(\.)?)|(ct(\.)?)|(dr(\.)?))\b'
address_pattern = Pattern(name="address", regex=address_regex, score=0.5)
address_recognizer = PatternRecognizer(supported_entity="ADDRESS_CUSTOM", patterns = [address_pattern], context=["st", "Apt"], deny_list=DENY_LIST_ADDRESS)

email_regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
email_pattern = Pattern(name="email address", regex=email_regex, score=0.5)
email_recognizer = PatternRecognizer(supported_entity="EMAIL_CUSTOM", patterns = [email_pattern], deny_list=DENY_LIST_EMAIL)

url_regex = r'https?://\S+|www\.\S+'
url_pattern = Pattern(name="url", regex=url_regex, score=0.5)
url_recognizer = PatternRecognizer(supported_entity="URL_CUSTOM", patterns = [url_pattern], deny_list=DENY_LIST_URL)

phone_recognizer = PhoneRecognizer(context=['phone', 'number', 'telephone', 'cell', 'cellphone', 'mobile', 'call', 'ph', 'tel', 'mobile', 'Email'])

In [11]:
#Credit to Microsoft for the Numbers Recognizer, which can identify numbers within words like "Fifty":
#https://microsoft.github.io/presidio/tutorial/03_rule_based/

class NumbersRecognizer(EntityRecognizer):

    expected_confidence_level = 0.7  # expected confidence level for this recognizer

    def load(self) -> None:
        """No loading is required."""
        pass

    def analyze(self, text: str, entities: list[str], nlp_artifacts: NlpArtifacts) -> list[RecognizerResult]:
        """
        Analyzes test to find tokens which represent numbers (either 123 or One Two Three).
        """
        results = []

        # iterate over the spaCy tokens, and call `token.like_num`
        for token in nlp_artifacts.tokens:
            if token.like_num:
                result = RecognizerResult(
                    entity_type="NUMBER",
                    start=token.idx,
                    end=token.idx + len(token),
                    score=self.expected_confidence_level,
                )
                results.append(result)
        return results

new_numbers_recognizer = NumbersRecognizer(supported_entities=["NUMBER"])

In [12]:
#More credit to Microsoft for the engine configs: https://microsoft.github.io/presidio/analyzer/customizing_nlp_models/

configuration = {
    "nlp_engine_name": "spacy",
    "models": [{"lang_code": "en", "model_name": "en_core_web_lg"}],
}
provider = NlpEngineProvider(nlp_configuration=configuration)
nlp_engine = provider.create_engine()

In [13]:
registry = RecognizerRegistry()
registry.load_predefined_recognizers()
registry.add_recognizer(address_recognizer)
registry.add_recognizer(email_recognizer)
registry.add_recognizer(url_recognizer)
registry.add_recognizer(new_numbers_recognizer)
registry.add_recognizer(phone_recognizer)

In [14]:
analyzer = AnalyzerEngine(supported_languages=['en'],
                          registry=registry,
                          nlp_engine=nlp_engine,
                          context_aware_enhancer=LemmaContextAwareEnhancer(
                              context_similarity_factor=0.6,
                              min_score_with_context_similarity=0.4
                          ))

## Training/Testing

In [15]:
preds = []

In [16]:
for i in tqdm(range(len(test))):
    start, end = tokens2index(test[i])
    test[i]['start'] = start
    test[i]['end'] = end

  0%|          | 0/10 [00:00<?, ?it/s]

In [17]:
for i, d in enumerate(tqdm(test)):
    results = analyzer.analyze(text=d['full_text'],
                               entities=["PHONE_NUMBER","PERSON","URL_CUSTOM","EMAIL_ADDRESS",
                                         "EMAIL_CUSTOM","ADDRESS_CUSTOM","US_SSN", "US_ITIN",
                                         "US_PASSPORT", "US_BANK_NUMBER","USERNAME"],
                               allow_list=ALLOW_LIST,
                               language='en', 
                               score_threshold=0.005)
    pre_preds = []
    for r in results:
        s = find_or_next_larger(d['start'], r.start)
        end = r.end
        word = d['full_text'][r.start:r.end]
        end = end - count_trailing_whitespaces(word)
        temp_preds = [s]
        try:
            while d['end'][s+1] <= end:
                temp_preds.append(s+1)
                s +=1
        except:
            pass
        
        tmp = False
        
        if r.entity_type == 'PHONE_NUMBER':
            if is_valid_date(word):
                continue
            for w in ALLOW_LIST:
                if w in d['full_text'][max(r.start-50, 0):min(r.end+50, len(d['full_text']))]:
                    tmp = False
                    break
                else:
                    tmp = True 
            label =  'PHONE_NUM'
        if r.entity_type == 'PERSON':
            label =  'NAME_STUDENT'
        if r.entity_type == 'URL_CUSTOM':
            for w in DENY_LIST_LEON:
                if w in word:
                    tmp = True
                    break
            label = 'URL_PERSONAL'
        if r.entity_type == 'EMAIL_ADDRESS' or r.entity_type == 'EMAIL_CUSTOM':
            label = "EMAIL"
        if r.entity_type == 'ADDRESS_CUSTOM':
            label = 'STREET_ADDRESS'
        if r.entity_type in ['US_SSN', 'US_ITIN', 'US_PASSPORT', 'US_BANK_NUMBER']:
            label = 'ID_NUM'
        if r.entity_type == 'USERNAME':
            label =  'USERNAME'
        if tmp:
            continue
        for p in temp_preds:
            if len(pre_preds) > 0:
                if pre_preds[-1]['rlabel'] == r.entity_type and ((p - pre_preds[-1]['token'])==1):
                    label_f = "I-"+label
                else:
                    label_f = "B-"+label
            else:
                label_f = "B-"+label
            pre_preds.append(({
                    "document":d['document'],
                    "token":p,
                    "label":label_f,
                    "rlabel":r.entity_type
                }))
    preds.extend(pre_preds)

  0%|          | 0/10 [00:00<?, ?it/s]

# Submission

In [18]:
submission = pd.DataFrame(preds).iloc[:,:-1].reset_index()
submission.columns = ['row_id','document', 'token', 'label']
submission

,row_id,document,token,label
0,0,7,39,B-STREET_ADDRESS
1,1,7,73,B-STREET_ADDRESS
2,2,7,116,B-STREET_ADDRESS
3,3,7,117,I-STREET_ADDRESS
4,4,7,138,B-STREET_ADDRESS
...,...,...,...,...
231,231,123,1657,I-NAME_STUDENT
232,232,123,1658,I-NAME_STUDENT
233,233,123,1549,B-URL_PERSONAL
234,234,123,1690,B-URL_PERSONAL


In [19]:
submission.to_csv('submission.csv', index = False)